In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Joao\AppData\Local\Temp\ipykernel_3212\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Construção do pipeline

A cosntrução deve ser feita com as seguintes etapas:
- deve ser separado a variável alvo das demais variáveis
- Os dados devem ser dividos em dados de treinamento e de teste. Isto é importante para a verificação do pipeline. Fazendo esta divisão é possivel comparar os resultados dos modelos treinados sem a utilização do pipeline (modelos as tranformações foram feitas no dado como um todo)*** com os resultado obtidos com a utilização do pipeline.
- Deve ser feita a transformação na variável separadamento dos métodos do pipeline. O pipeline só vai alterar as variáveis de treinamento
- Aplica o pipeline
- Usando este método no final do processo deve ser feita uma operação inversa da aplicada na variável alvo para alerar os resultados obtidos chagando na classificação desejada 


***Obs: em pipeline as tranformações não são feitas nos dados como um todo. É feita a divisão dos dados em dados de teste e de treinameto. Como o método fit o pipeline transforma os dados de treinamento e treina o modelo (nesta etapa nada é feito nos dados de teste). Quando o pipeline é usado para previsões nos dados de teste é que são feitas as transformações nos dados de teste e em sequência é aplicado o modelo treinado.

In [6]:
#Biblioteca

#Métodos prontos na biblioteca feature-engine
#Ref.: -https://feature-engine.readthedocs.io/en/latest/index.html
#      - arquivo 7 do curso do Udemy


# Imputation (adicionar valor to NaN)
#===================
# -AddMissingIndicator
# -MeanMedianImputer
# -CategoricalImputer

# Transformation
#===================
# -LogTransformer
# -YeoJohnsonTransformer

# Encoding
#===================
# -RareLabelEncoder
# -OrdinalEncoder

# Selection
#===================
# -DropFeatures

# Exemplo de transformacao usando a biblioteca feature-engine
# YeoYohnson_trans = YeoJohnsonTransformer(variables = CONTINUOUS_VAR)
# YeoYohnson_trans.fit(df)
# df=YeoYohnson_trans.transform(df).head(3)


In [5]:
import pandas as pd
import numpy as np

#da biblioteca feature-engine
from feature_engine.encoding import OneHotEncoder
from feature_engine.transformation import YeoJohnsonTransformer

#para transformações
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import Normalizer

# metodos sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics

# modelos
from sklearn.ensemble import RandomForestClassifier

# para salvar o modelo
import joblib

In [6]:
#leitura dos dados
df = pd.read_csv("dados\\dados_treinamento.csv" ,sep=',');

# linhas e colunas
print(df.shape)

# visualiza o dado
df.head()

(10847, 12)


,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,35.0,M,173.6,84.08,25.4,97.0,157.0,48.8,26.4,58.0,229.0,B
1,22.0,F,168.3,56.30,20.9,74.0,117.0,27.4,21.5,56.0,192.0,A
2,59.0,F,152.9,55.20,33.9,68.0,142.0,27.4,15.3,15.0,146.0,C
3,30.0,F,153.3,54.60,26.7,75.0,127.0,25.7,19.1,37.0,152.0,C
4,23.0,F,167.1,58.80,28.0,60.0,104.0,28.9,19.8,45.0,166.0,A


## Preparação dos dados

In [7]:
#seleciona variaveis de treinamento e alvo
var_train=[var for var in df.columns if var != 'class'] 

X = df[var_train]
y = df['class']

In [8]:
#Divide os dados
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.1,
                                                    random_state=0)

X_train.shape, X_test.shape

((9762, 11), (1085, 11))

## Variável alvo

Nesta seção fazer as alterações necessárias na variável alvo

In [9]:
#altera a variável alvo
class_map = {'C':0, 'D':1, 'A':2, 'B':3 }
y_train= y_train.map(class_map)
                          
# Obs: 
# -poderia ter alterado antes da divisão dos dados, assim ja ficaria
#  alterado para os valores de teste também. Porém eu optei por fazer 
#  desta forma pora ficar mais parecedo com o que seria o caso real.
# -as tranformções da variável alvo não funciona no pipeline, pois
#  esta variável não possui nome na coluna (está escrita como nparray),
#  mas também não faz sentido colocar a variável alvo no pipeline, pois
#  não teremos acesso a esta variável numa aplicação do modelo, já que esta
#  informação é a que estamos querendo prever, ela não é acessível ao usuário.

## Classes construidas

In [10]:
# Classes para mapeamentos
#=============================

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin


class Mapeamento(BaseEstimator, TransformerMixin):    

    def __init__(self, variavel, mapa):
        #Parametros
        # -mapas -->lista com dicionários(mapas)
        self.variavel = variavel
        self.mapa = mapa 

    def fit(self, X, y=None):
        return self

    def transform(self, X):        
        X = X.copy()        
        X[self.variavel] = X[self.variavel].map(self.mapa)
        return X


## Configurações

In [11]:
# Definições gerais
gender_map = {'M': 0, 'F': 1}

CONTINUOUS_VAR=['age','height_cm', 'weight_kg', 'body fat_%', 'diastolic',
                 'systolic', 'gripForce', 'sit and bend forward_cm',
                 'sit-ups counts', 'broad jump_cm']

## Pipeline

In [12]:
# Cosntrução do pipeline
model_pipe= Pipeline([     
    ('YeoYohnson_trans',YeoJohnsonTransformer(variables = CONTINUOUS_VAR)),
    ('map_gender',Mapeamento(variavel='gender',mapa=gender_map)),       
    ('randForest', RandomForestClassifier(n_estimators=50,
                             max_depth=24,
                             max_features=5,
                             bootstrap=True,
                             max_samples=None,
                             random_state=0))
])

In [13]:
# Fita o pipeline
model_pipe.fit(X_train,y_train)

Pipeline(steps=[('YeoYohnson_trans',
                 YeoJohnsonTransformer(variables=['age', 'height_cm',
                                                  'weight_kg', 'body fat_%',
                                                  'diastolic', 'systolic',
                                                  'gripForce',
                                                  'sit and bend forward_cm',
                                                  'sit-ups counts',
                                                  'broad jump_cm'])),
                ('map_gender',
                 Mapeamento(mapa={'F': 1, 'M': 0}, variavel='gender')),
                ('randForest',
                 RandomForestClassifier(max_depth=24, max_features=5,
                                        n_estimators=50, random_state=0))])

In [14]:
# Avaliação do modelo
#===========================

# IMPORTANTE: os resultados devem bater com os resultados
# obtidos no arquivo de treinamento do modelo sem o pipeline

# modelos previstos
y_hat = model_pipe.predict(X_test)

# transformação dos dados de teste
class_map = {'C':0, 'D':1, 'A':2, 'B':3 }
y_test= y_test.map(class_map)


print('Valores previstos: ', y_hat[0:10])
print('Valores reais: ', y_test[0:10].to_list())
print("Acuracia:", metrics.accuracy_score(y_test, y_hat))

Valores previstos:  [1 2 2 0 3 0 1 1 3 0]
Valores reais:  [1, 2, 2, 0, 3, 0, 1, 1, 3, 0]
Acuracia: 0.7290322580645161


In [15]:
# salva o pipeline
joblib.dump(model_pipe, 'model_pipe.joblib')

['model_pipe.joblib']

## Score new datas

In [17]:
f_teste = joblib.load('model_pipe.joblib') 